# CIS(D)

In [13]:
import numpy as np
import pyci
import psi4

In [14]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('6-31g*', '../.xyz/LiCN.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('.scratch/ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('.scratch/mo_scf_info.npz')['eps_a']
Ca = np.load('.scratch/mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints, greedy=True)
del ao_erints

Size of the ERI tensor will be 0.03 Gb

Ground state SCF Energy : -99.77249855 

Nuclear repulsion energy : 27.84883421 

Total electronic energy : -127.62133275 

Using greedy eri_ao2mo transform



In [15]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 45  
Molecular orbitals: 45  
Spin Orbitals     : 90  
Alpha Orbitals    : 8  
Beta Orbitals     : 8  
Occupied Orbitals : 8  
Virtual Orbitals  : 37  



In [16]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
singles = True
full_cis = True
doubles = False
doubles_iiaa = False
doubles_iiab = False
doubles_ijaa = False
doubles_ijab_A = False
doubles_ijab_B = False
options = [singles, full_cis, doubles,
           doubles_iiaa, doubles_iiab,doubles_ijaa,
           doubles_ijab_A, doubles_ijab_B]
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)


([1, 296, 0, 0, 0, 0, 0], 297)

In [17]:
scf_energy = aoint.scf_energy

In [18]:
HCIS = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=2)

In [19]:
HCIS0 = HCIS - np.eye(sum(num_csfs))*scf_energy
ECIS, CCIS = np.linalg.eigh(HCIS0)

In [8]:
mo_so_erints = aoint.eri_mo2so_psi4(Ca, Ca)
mo_so_erints = mo_so_erints.transpose(0,2,1,3)
cis_d_class = pyci.configint.CIS_D(ECIS, CCIS, eps_a, mo_so_erints, (nocc, nmo))

In [9]:
# MP2 energy calculation
print('\nComputing MP2 corrections\n')
mp2_e = psi4.energy('mp2')
print('PSI4 MP2 Ground state energy: %3.16f' % mp2_e)
print('CALC MP2 Ground state energy: %3.16f' % (scf_energy+cis_d_class.comp_e0mp2()))
print('Difference : %3.2E'%(scf_energy+cis_d_class.comp_e0mp2() - mp2_e))



Computing MP2 corrections

PSI4 MP2 Ground state energy: -100.0658987157554378
CALC MP2 Ground state energy: -100.0658202048036145
Difference : 7.85E-05


In [10]:
e_cis_d = cis_d_class.comp_cis_d(50, ncores=4)

Ei_d_corr 	 Ei_t_corr 	 Ei_corr 	 Ei_cis(d)
-0.0000 	 0.0000 	 0.0000 	 0.0000
-0.0704 	 0.1044 	 0.0340 	 0.2730
-0.0949 	 0.0785 	 -0.0165 	 0.2253
-0.0949 	 0.0785 	 -0.0165 	 0.2253
-0.0752 	 0.1020 	 0.0268 	 0.2747
-0.0752 	 0.1020 	 0.0268 	 0.2747
-0.1076 	 0.0481 	 -0.0595 	 0.2035
-0.1105 	 0.0674 	 -0.0431 	 0.2460
-0.1105 	 0.0674 	 -0.0431 	 0.2460
-0.0972 	 0.0751 	 -0.0221 	 0.2807
-0.0831 	 0.0901 	 0.0070 	 0.3372
-0.0806 	 0.0804 	 -0.0003 	 0.3321
-0.0806 	 0.0804 	 -0.0003 	 0.3321
-0.0828 	 0.0937 	 0.0110 	 0.3464
-0.0828 	 0.0937 	 0.0110 	 0.3464
-0.1030 	 0.0744 	 -0.0286 	 0.3406
-0.1030 	 0.0744 	 -0.0286 	 0.3406
-0.1184 	 0.0524 	 -0.0660 	 0.3165
-0.1827 	 0.0535 	 -0.1292 	 0.2631
-0.1381 	 0.0576 	 -0.0805 	 0.3371
-0.1381 	 0.0576 	 -0.0805 	 0.3371
-0.1034 	 0.0731 	 -0.0303 	 0.3928
-0.0986 	 0.0755 	 -0.0231 	 0.4094
-0.0986 	 0.0755 	 -0.0231 	 0.4094
-0.0950 	 0.0799 	 -0.0151 	 0.4340
-0.0947 	 0.0809 	 -0.0137 	 0.4357
-0.0947 	 0.0809 	 -0.0137 

In [20]:
matrix_eig = np.einsum('iA, AB, Bj-> ij', CCIS.T, HCIS, CCIS, optimize=True)


NameError: name 'vecs' is not defined